In [38]:
import pandas as pd
import numpy as np
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from tqdm import tqdm  # Visualize loop progress
# from sklearn.linear_model import LinearRegression
from tenacity import retry, stop_after_attempt, wait_fixed
import openpyxl
import os
import requests
import yfinance as yf

import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

In [39]:
# tradingview lightweight-charts
from lightweight_charts import Chart
import asyncio
import nest_asyncio
# Apply nest_asyncio AT START to allow nested event loops (required for Jupyter)
nest_asyncio.apply()
#
from dotenv import load_dotenv

In [40]:
# load env vars
load_dotenv()

True

In [41]:
# #####---------------------#####s
# screen_date = '2025-03-07'
# screen_date = '2025-03-13'
# screen_date = '2025-03-19'
# screen_date = '2025-03-20'
screen_date = '2025-03-24'
# #####---------------------#####
# read pre market data from TradingView data via csv
trading_view_df = pd.read_csv(f"/Users/sudz4/Desktop/MATT-MATTHEW/matt-matthew/tv_screen_gap-up_{screen_date}.csv")

In [42]:
# print(trading_view_df.columns)

In [43]:
# CREATE CATEGORIES FOR MARKET CAP
def categorize_market_cap(df):
    # categorize stocks groups by market cap
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    # marekt cap categories list
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df
# execute categorization
category_setup_df = categorize_market_cap(trading_view_df).copy()
# drop Undefined marketCapType
category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume',
    'Exponential Moving Average (3) 1 day', 'Exponential Moving Average (7) 1 day'
]

# apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

# CRITERIA CONFIGURATION FOR EACH MARKET CAP CATEGORY
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

# FILTER STOCKS BASED ON CONFIGURATION CRITERIA - STAGING
def filter_stocks(df, config):
    # stock filtering conditions >=< based on criteria
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

# SCREEN STOCKS BY CATEGORY
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# EXECUTE KEY SCREENING FUNCTION AND CREATE DATAFRAME
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# rendered column list and ordering
cols_list = [
    'Symbol', 
    'Description', 
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes',
    'Exponential Moving Average (3) 1 day',
    'Exponential Moving Average (7) 1 day'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Pre-market Change %', 'Price'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stocks found.")
display(smash_df.head(20))
# display(smash_df)
# # ALL SMASH
# display(smash_df)

55 stocks found.


,Symbol,Description,marketCapType,Pre-market Change %,Pre-market Gap %,Market capitalization,Price,Pre-market Open,Industry,Index,Sector,Exchange,Recent earnings date,Upcoming earnings date,Float shares outstanding,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Relative Volume 1 day,Relative Volume 5 minutes,Relative Volume 30 minutes,Relative Volume at Time,Analyst Rating,Technical Rating 5 minutes,Exponential Moving Average (3) 1 day,Exponential Moving Average (7) 1 day
0,SKT,Tanger Inc.,Mid caps,6.649937,6.649937,3.593027e+09,31.88,34.00,Real estate investment trusts,"Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000",Finance,NYSE,2025-02-19,2025-04-30,1.098580e+08,998630.1,8.871403e+05,7.752794e+05,1.421824,8.512194,5.784928,1.128666,Buy,Sell,32.115133,32.363118
1,TEM,"Tempus AI, Inc.",Mid caps,6.095424,1.265823,8.843121e+09,51.35,52.00,Packaged software,"NASDAQ Composite, Nasdaq US Mid Cap Growth, NASDAQ Biotechnology",Technology services,NASDAQ,2025-02-24,2025-05-28,7.799817e+07,10234143.0,1.557061e+07,9.472504e+06,0.975840,4.029848,2.872377,0.894373,Buy,Buy,49.532172,48.559044
2,KC,Kingsoft Cloud Holdings Limited,Mid caps,6.027220,6.804925,4.125302e+09,15.43,16.48,Data processing services,"NASDAQ Composite, NASDAQ Golden Dragon China, NASDAQ Computer",Technology services,NASDAQ,2025-03-19,2025-06-04,2.536844e+08,3778325.0,4.340737e+06,4.058844e+06,2.109071,5.325117,2.687931,2.095263,Buy,Buy,16.371914,16.959415
3,UPST,"Upstart Holdings, Inc.",Mid caps,4.022325,2.424942,4.869223e+09,51.96,53.22,Internet software/Services,"NASDAQ Composite, Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000, NASDAQ Real Estate and Other Financial Services, KBW NASDAQ Financial Technology Index",Technology services,NASDAQ,2025-02-11,2025-05-13,8.131266e+07,5212172.2,7.507890e+06,6.680660e+06,0.822649,8.735979,3.762548,0.703912,Neutral,Buy,50.836879,50.512613
4,WF,Woori Financial Group Inc.,Mid caps,3.721067,3.721067,8.478289e+09,34.13,35.40,Major banks,NaN,Finance,NYSE,2025-02-07,2025-04-17,2.464940e+08,45959.5,6.037163e+04,6.112681e+04,0.813436,0.736797,0.615498,0.754537,Buy,Strong buy,34.058003,33.877942
5,RH,RH,Mid caps,3.641919,2.093484,4.505238e+09,242.18,247.25,Specialty stores,"S&P MidCap 400, Russell 3000, Russell 1000",Retail trade,NYSE,2024-12-12,2025-03-26,1.511141e+07,1195246.1,8.865320e+05,7.112258e+05,1.036636,11.295165,4.410527,0.937320,Buy,Neutral,236.356874,235.836719
6,GLBE,Global-E Online Ltd.,Mid caps,3.323661,3.323661,6.330674e+09,37.91,39.17,Internet retail,"NASDAQ Composite, NASDAQ Computer",Retail trade,NASDAQ,2025-02-19,2025-05-26,1.027973e+08,2032580.0,2.184248e+06,1.651721e+06,0.774681,7.253512,6.342850,0.674792,Strong buy,Strong buy,37.418152,37.374281
7,W,Wayfair Inc.,Mid caps,3.241236,1.188354,4.271606e+09,33.66,34.06,Specialty stores,"Nasdaq US Small Cap Growth, Russell 3000, Russell 1000",Retail trade,NYSE,2025-02-20,2025-05-01,9.277741e+07,4255679.7,4.532743e+06,3.972603e+06,1.324797,9.843005,6.173269,1.374134,Buy,Strong buy,32.564452,32.393644
8,DNB,"Dun & Bradstreet Holdings, Inc.",Mid caps,3.092784,0.114548,3.854438e+09,8.73,8.74,Packaged software,"Russell 3000, Russell 1000",Technology services,NYSE,2025-02-20,2025-05-01,3.073450e+08,5187977.3,4.911358e+06,3.391556e+06,2.876351,2.519605,3.718317,3.235989,Buy,Buy,8.605631,8.486711
9,NCLH,Norwegian Cruise Line Holdings Ltd.,Mid caps,2.941176,0.833333,8.974874e+09,20.40,20.57,Hotels/Resorts/Cruise lines,"S&P 500, S&P 500 Consumer Discretionary, Nasdaq US Mid Cap Growth, Russell 3000, Russell 1000, S&P 500 ESG",Consumer services,NYSE,2025-02-27,2025-05-06,4.349908e+08,17236295.9,1.535431e+07,1.107842e+07,1.169853,5.970850,2.259555,0.895961,Buy,Sell,20.114408,19.883420


In [44]:
# group and count by sector to new df called sector_count_df
sector_count_df = smash_df.groupby('Sector').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
# print the date and time of the screen
print(f"Screening date: {screen_date}")
print(f"Total stocks returned: {len(smash_df)}")
display(sector_count_df)

Screening date: 2025-03-24
Total stocks returned: 55


,Sector,Count
4,Finance,11
10,Technology services,11
2,Consumer services,7
1,Consumer non-durables,6
9,Retail trade,6
6,Health technology,3
3,Electronic technology,2
7,Process industries,2
8,Producer manufacturing,2
11,Transportation,2


In [45]:
smash_list = smash_df['Symbol'].tolist()
print(len(smash_list))
print(smash_list)

55
['SKT', 'TEM', 'KC', 'UPST', 'WF', 'RH', 'GLBE', 'W', 'DNB', 'NCLH', 'PTON', 'GENI', 'JOBY', 'AEO', 'NFE', 'ASAN', 'QFIN', 'CZR', 'WGS', 'CAR', 'VFC', 'FG', 'CPRX', 'OLLI', 'PK', 'KBH', 'IQ', 'PVH', 'SKX', 'HBI', 'VIAV', 'SPSC', 'RRR', 'APPN', 'CC', 'BRZE', 'FIVE', 'CROX', 'APLE', 'IFS', 'TRMK', 'LEVI', 'KNX', 'CXT', 'SPR', 'WH', 'VAC', 'SMG', 'CATY', 'TNL', 'RRX', 'CYTK', 'ITGR', 'HUBG', 'AKR']


In [46]:
# Create a filter for EMA3 > EMA7
smash_ema_df = smash_df['Exponential Moving Average (3) 1 day'] > smash_df['Exponential Moving Average (7) 1 day']

# Apply the filter to smash_df
filtered_smash_df = smash_df[smash_ema_df]

# Print the number of stocks that meet the criteria
print(f"Number of stocks where EMA3 > EMA7: {len(filtered_smash_df)}")
print("\nFiltered stocks:")
print(filtered_smash_df['Symbol'].tolist())

Number of stocks where EMA3 > EMA7: 28

Filtered stocks:
['TEM', 'UPST', 'WF', 'RH', 'GLBE', 'W', 'DNB', 'NCLH', 'PTON', 'GENI', 'JOBY', 'AEO', 'NFE', 'QFIN', 'WGS', 'CPRX', 'OLLI', 'HBI', 'VIAV', 'SPSC', 'APPN', 'BRZE', 'FIVE', 'CROX', 'IFS', 'SPR', 'RRX', 'CYTK']
